<a href="https://colab.research.google.com/github/evansnjagi/spacex_modeling/blob/main/SpaceX_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# getting the data from the spaceX api
# Objectives
- Booster version  from the rocket database
- Launch site and get, latitude, longitude, and the name
- Payload getting the mass kilograms and the orbit
- core(the outcome for landing). We will get the following data, type of landing, number of flights with the core, whether grids were used, whether the core we re-used, whether legs were used, landing pad used, block of the core( number used to separate version cores), number of times the specific core has been reused and finally serial of the core.

In [3]:
%%capture
! pip install requests

In [4]:
# library codes

import requests
import pandas as pd
import json


In [5]:
# getting the entire dataset
response = requests.get("https://api.spacexdata.com/v4/launches/past")
data = pd.json_normalize(response.json())
print(type(data))
print(data.shape)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
(187, 43)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   static_fire_date_utc       121 non-null    object 
 1   static_fire_date_unix      121 non-null    float64
 2   net                        187 non-null    bool   
 3   window                     117 non-null    float64
 4   rocket                     187 non-null    object 
 5   success                    186 non-null    object 
 6   failures                   187 non-null    object 
 7   details                    134 non-null    object 
 8   crew                       187 non-null    object 
 9   ships                      187 non-null    object 
 10  capsules                   187 non-null    object 
 11  payloads                   187 non-null    object 
 12  launchpad                  187 non-null    object 
 13  fl

In [6]:
# booster version
rocket = data["rocket"]
BoosterVersion = []

for rocket in rocket:
    if rocket:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(rocket)).json()
        BoosterVersion.append(response["name"])
# boosterversion into a series

BoosterVersion = pd.Series(BoosterVersion)

In [7]:
print(type(BoosterVersion))
print(f"The length of the booster version is: {len(BoosterVersion)}")

<class 'pandas.core.series.Series'>
The length of the booster version is: 187


In [8]:
from re import L
# launchpad, latitude, longitude, and the name of the lauchpad
launchpad = data["launchpad"]
LaunchSite = []
Latitude = []
Longitude = []

for launchpad in launchpad:
    if launchpad:
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(launchpad)).json()
        latitude = response["latitude"]
        longitude = response["longitude"]
        name = response["name"]
        LaunchSite.append(name)
        Latitude.append(latitude)
        Longitude.append(Longitude)
# into a series
LaunchSite = pd.Series(LaunchSite)
Latitude = pd.Series(Latitude)
Longitude = pd.Series(Longitude)

In [9]:
print(type(LaunchSite))
print(f"The lenght of the launch site is: {len(LaunchSite)}")
print(type(Latitude))
print(f"The lenght of the Latitude is: {len(Latitude)}")
print(type(Longitude))
print(f"The length of the longitude is: {len(Longitude)}")

<class 'pandas.core.series.Series'>
The lenght of the launch site is: 187
<class 'pandas.core.series.Series'>
The lenght of the Latitude is: 187
<class 'pandas.core.series.Series'>
The length of the longitude is: 187


In [10]:
# getting payload data, mass and orbit
Payloads = pd.Series(data["payloads"])
Payload = Payloads.apply(lambda x: x[0])

Mass_kg = []
Orbit = []

for payload in Payload:
    if payload:
        response = requests.get("https://api.spacexdata.com/v4/payloads/" + str(payload)).json()
        Mass_kg.append(response["mass_kg"])
        Orbit.append(response["orbit"])
Mass_kg = pd.Series(Mass_kg)
Orbit = pd.Series(Orbit)

In [11]:
print(f"The length of the mass is: {len(Mass_kg)}")
print(type(Mass_kg))
print(f"The lenght of the orbit is: {len(Orbit)}")
print(type(Orbit))

The length of the mass is: 187
<class 'pandas.core.series.Series'>
The lenght of the orbit is: 187
<class 'pandas.core.series.Series'>


In [12]:
# Initialize lists
Block = []
ReusedCount = []
Serial = []
Outcome = []
Flights = []
GridFins = []
Legs = []
Reused = []
LandingPad = []

# Assuming 'data' is your DataFrame
cores = data["cores"]
Cores = cores.apply(lambda x: x[0] if len(x) > 0 else None)  # Handling empty lists

for core in Cores:
    if core:
        core_series = pd.Series(core)
        Core = core_series["core"]

        # Requesting the outcome of landing data
        response = requests.get("https://api.spacexdata.com/v4/cores/" + str(Core)).json()

        Block.append(response["block"])
        ReusedCount.append(response["reuse_count"])
        Serial.append("serial")

        Outcome.append(str(response.get('landing_success', 'None')) + ' ' + str(response.get('landing_type', 'None')))
        Flights.append(core_series.get('flight', None))
        GridFins.append(core_series.get('gridfins', None))
        Reused.append(core_series.get('reused', None))
        Legs.append(core_series.get('legs', None))
        LandingPad.append(core_series.get('landpad', None))
    else:
        Block.append(None)
        ReusedCount.append(None)
        Serial.append(None)
        Outcome.append(None)
        Flights.append(None)
        GridFins.append(None)
        Legs.append(None)
        Reused.append(None)
        LandingPad.append(None)

# Convert lists to series
Block = pd.Series(Block)
ReusedCount = pd.Series(ReusedCount)
Serial = pd.Series(Serial)
Outcome = pd.Series(Outcome)
Flights = pd.Series(Flights)
GridFins = pd.Series(GridFins)
Legs = pd.Series(Legs)
Reused = pd.Series(Reused)
LandingPad = pd.Series(LandingPad)


In [13]:
print(f"The length of block is {len(Block)}")
print(type(Block))
print(f"The length of reused count is {len(ReusedCount)}")
print(type(ReusedCount))
print(f"The length of serial is {len(Serial)}")
print(type(Serial))
print(f"The length of outcome is {len(Outcome)}")
print(type(Outcome))
print(f"The length of flights is {len(Flights)}")
print(type(Flights))
print(f"The length of grid fins is {len(GridFins)}")
print(type(GridFins))
print(f"The length of legs is {len(Legs)}")
print(type(Legs))
print(f"The length of reused is {len(Reused)}")
print(type(Reused))
print(f"The length of landing pads is {len(LandingPad)}")
print(type(LandingPad))

The length of block is 187
<class 'pandas.core.series.Series'>
The length of reused count is 187
<class 'pandas.core.series.Series'>
The length of serial is 187
<class 'pandas.core.series.Series'>
The length of outcome is 187
<class 'pandas.core.series.Series'>
The length of flights is 187
<class 'pandas.core.series.Series'>
The length of grid fins is 187
<class 'pandas.core.series.Series'>
The length of legs is 187
<class 'pandas.core.series.Series'>
The length of reused is 187
<class 'pandas.core.series.Series'>
The length of landing pads is 187
<class 'pandas.core.series.Series'>


In [14]:
# creating a dictionary

launch_dict = {
    'FlightNumber': list(data['flight_number']),
    'Date': list(data['date_utc']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass': Mass_kg,
    'LaunchSite': LaunchSite,
    'Orbit': Orbit,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    "Reused": Reused,
    'ReusedCount': ReusedCount,
    'Block': Block,
    'Serial': Serial,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Latitude': Latitude,
    'Longitude': Longitude,
}

In [15]:
# creating a dataframe
df = pd.DataFrame(launch_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    187 non-null    int64  
 1   Date            187 non-null    object 
 2   BoosterVersion  187 non-null    object 
 3   PayloadMass     162 non-null    float64
 4   LaunchSite      187 non-null    object 
 5   Orbit           186 non-null    object 
 6   Outcome         187 non-null    object 
 7   Flights         187 non-null    int64  
 8   GridFins        187 non-null    bool   
 9   Reused          187 non-null    bool   
 10  ReusedCount     187 non-null    int64  
 11  Block           182 non-null    float64
 12  Serial          187 non-null    object 
 13  Legs            187 non-null    bool   
 14  LandingPad      151 non-null    object 
 15  Latitude        187 non-null    float64
 16  Longitude       187 non-null    object 
dtypes: bool(3), float64(3), int64(3), o

In [16]:
# getting total number of null values
df.isnull().sum()

,0
FlightNumber,0
Date,0
BoosterVersion,0
PayloadMass,25
LaunchSite,0
Orbit,1
Outcome,0
Flights,0
GridFins,0
Reused,0


In [17]:
# filling the payload mass with the mean
mean = df["PayloadMass"].mean().round(2)
df["PayloadMass"] = df["PayloadMass"].fillna(mean)

In [18]:
df.isnull().sum()

,0
FlightNumber,0
Date,0
BoosterVersion,0
PayloadMass,0
LaunchSite,0
Orbit,1
Outcome,0
Flights,0
GridFins,0
Reused,0


In [19]:
data_falcon9 = df[df['BoosterVersion'] == 'Falcon 9']

In [22]:
# saving the spaceX data
path = "spaceX_falcom9.csv"
data_falcon9.to_csv(path)

# AUTHERED BY: EVANS NJAGI KARAGO